crsid: oea27

[tk - I need to check for typos and update citations; maybe don't number citations?]

[tk - number the sections?]

[search for all the figures and make sure they're labelled correctly]

[update image captions and alts.]

# Retraining-free Methods for Optimizing Deep Neural Networks 

Many state-of-the-art deep neural networks (DNNs) are large models with up to billions of parameters, which must be optimized for efficient deployment on edge devices. Optimization techniques include quantization, using knowledge distillation to train a smaller model, pruning, and so on.  [tk - is this accurate?]. Often, these methods require a form of retraining or finetuning to retain the desired accuracy. In this project, I explore techniques for optimizing a DNN that do not require any  training or finetuning. The aim is to investigate the performance gains we can attain for a pre-trained model with training data available.

To do this, I built ONNXSat, a tool that accepts a pre-trained ONNX model and optimizes it to reduce its size and inference latency. Using ONNXSat, I investigate retraining-free optimization techniques on convolutional neural networks (CNNs) by conducting experiments to answer three questions:

1. What is the effect of operator fusion on a model's performance?  
2. How much sparsity can we introduce into a model while keeping accuracy loss within a threshold?
3. How does sparsity affect a model's performance? - [tk - I might need to update this]


[Results show that...tk: fill out]


**Note**: The cells in this report are not executable, as importing all the logic used in ONNXSat would have made this writeup large. You can verify the code by running `python3 -m bench.bench <experiment_no> --d [optional results directory]` from the root directory to execute any benchmark. Alternatively, you can also run `python3 main.py` to test the end-to-end flow, or `make test` to execute the conversion tests.

## 1. Equality Saturation for Operator Fusion
Operator fusion is a common optimization technique used in ML compilers to improve model performance by reducing the number of computations during inference. In this experiment, I evaluate the impact of operator fusion on model performance by implementing operator fusion in ONNXSat using a technique known as _equality saturation_ [1].

Equality saturation addresses the phase-ordering problem in traditional compilers, where the order of applying optimizations affects the performance of a program. Borrowing the classic example from Willsey et al. [2], say a compiler has a rewrite rule stating that expressions of the form $x \times 2$ should be written to use the left shift operator: $x \ll 1$. 

If it receives an input expression $(a \times 2)/2$, it might rewrite the program to $(a \ll 2)/2$, which prevents simplifying the expression to $a$. The challenge here is that term rewriting in traditional compilers is destructive. After applying a rewrite, the original form is lost. 

Equality saturation uses an *equivalence graph* (e-graph) data structure to address the term rewriting problem. This e-graph contains *equivalence nodes* (e-nodes), which represent expressions in the program. Equivalent e-nodes belong to the same *equivalence class* (e-class), and edges point from e-nodes to child e-classes.

During an exploration phase, an equality saturation engine builds the e-graph by applying user-provided rewrite rules to create new e-nodes until the e-graph has reached *saturation*, where no more rules can be applied. The engine then extracts the best version of the program according to a user-provided cost model. 

Figure 1 illustrates the e-graph for the expression in the classic example. Boxes with dotted borders represent e-classes, while e-nodes have solid borders. As shown in the figure, the e-graph contains all equivalent forms of the program until all the rewrites have been applied. With the e-graph structure, the order in which rewrites are applied does not affect the quality of the program. 

Instead, the engine extracts the program with the minimum cost based on the costs of the different expressions. 

<figure style="text-align: center; margin: 5px">
  <img
  src="report/egraph.png"
  alt="Image containing three figures, each showing the resulting e-graph from applying rewrites to the input program."
  style="width: 50%; height: auto;">
  <figcaption> <b>Figure 1. </b> E-graph for optimizing (a * 2)/2. Boxes with dotted borders represent e-classes, which contain equivalent e-nodes represented with solid borders. </figcaption>
</figure>

Prior work by TENSAT [3] and MCTS [4] uses equality saturation in Rust to optimize DNN computation graphs by applying rewrites generated by TASO [5]. ONNXSat differs from these by incorporating fusions not expressed by either of them to the best of my knowledge, and does not rely on TASO for its rewrite rules. In addition, it extends these by also supporting pruning neural network weights.  


### 1.1 Building ONNXSat
Given a pre-trained model in ONNX format, ONNXSat applies equality saturation to perform operator fusion and generate an optimized version of the model. ONNXSat uses *Egglog* [6], a Python library, as its equality saturation engine. At a high level, ONNXSat:

- Converts an ONNX model to the Egglog format.
- Applies pre-defined rewrites rules on the model using Egglog and extracts the best expression.
- Converts the returned expression back into ONNX format for running inference. 
- Uses the ONNX Runtime Engine for inference.

This section describes the different components in more detail. 


#### 1.1.1. ONNX to Egglog Conversion
ONNXSat currently supports models containing 48 ONNX operators defined in [operators.py](eggie/operators.py).  For each operator, I created an equivalent method representing an Egglog e-node (or expression) according to the operator specification from ONNX. 

For example, the `MatMul` operator has a name, accepts two tensor inputs, $a$ and $b$, and produces an output. It is defined in ONNXSat as:

In [ ]:
@method()
@classmethod
def MatMul(cls, name: TensorId, a: TensorId, b: TensorId, output: TensorId) -> TensorId: ...

Some operators have attributes. For example, the `Conv` operator has attributes defined as:

In [ ]:
class ConvAttrs(Expr):
    @method()
    def __init__(
        self,
        auto_pad: String,
        group: i64,
        dilations: Vec[i64],
        kernel_shape: Vec[i64],
        pads: Vec[i64],
        strides: Vec[i64],
    ): ...

The `Conv` operator is then represented as:

In [ ]:
@method()
@classmethod
def Conv(cls, 
        name: TensorId, 
        attrs: ConvAttrs,
        x: TensorId,
        w: TensorId,
        b: TensorId,
        output: TensorId,
    ) -> TensorId: ... 


The `Op` class in [operators.py](eggie/operators.py) contains the definition of all the supported operators. It inherits from the Egglog `Expr` class, which is the base class for representing e-nodes.

Each operator receives and returns `TensorId` objects, also defined in [operators.py](eggie/operators.py), making it easier to interchange between operators when expressing rewrite rules. 

The full definition of [operators.py](eggie/operators.py) is omitted from this writeup for brevity, as it is nearly 500 lines long.  


The `OnnxParser` class defined in [onnx_e/parser.py](onnx_e/parser.py) contains methods to convert each supported ONNX operator node to its Egglog representation. 

For example, I define the method to convert an ONNX node representing a `MatMul` operation into the Egglog representation presented above as:

In [ ]:
def _convert_matmul(self, node: NodeProto) -> TensorId:
    return Op.MatMul(
        self._to_tensor_id(node.name),
        self._to_tensor_id(node.input[0]),
        self._to_tensor_id(node.input[1]),
        self._to_tensor_id(node.output[0]),
    )

Where `NodeProto` represents an ONNX operator and `_to_tensor_id()` either returns an existing `TensorId` object if the node has been processed before, or returns a new one:

In [ ]:
def _to_tensor_id(self, id: str) -> TensorId:
    if id in self._output_per_node:
        return self._output_per_node[id]

    return TensorId(id)

`ONNXParser` defines similar methods to `_convert_matmul()` for the supported operators, with additional processing for operators with attributes. The full class definition can be found in [onnx_e/parser.py](onnx_e/parser.py), containing approximately 800 lines. 



The conversion from ONNX to Egglog format is handled in the `Converter` class defined in [converter.py](converter.py) as:

In [ ]:
from eggie.operators import *
from eggie.parser import EggParser
from egglog import Expr
from onnx.onnx_ml_pb2 import GraphProto, NodeProto
from typing import List

from onnx_e.parser import OnnxParser


class Converter:
    @classmethod
    def to_egglog(cls, graph: GraphProto) -> Expr:
        return OnnxParser(graph).parse()

    @classmethod
    def to_onnx(cls, egglog: Expr) -> List[NodeProto]:
        parser = EggParser(egglog)
        return parser.parse()


Where `GraphProto` represents an ONNX computation graph. 


For an ONNX graph with the operator sequence: Conv -> Relu -> MaxPool, the resulting Egglog expression from the conversion logic has the form:

In [ ]:
Op.MaxPool(
    MaxPoolName,
    MaxPoolAttributes,
    Op.Relu(
        ReluName,
        Op.Conv(
            ConvName,
            ConvAttributes,
            ConvInput,
            ConvWeight,
            ConvBias,
            ConvOut,
        ),
        ReluOut,
    ),
    MaxPoolOut,
)

`Op.Relu` takes the result of `Op.Conv` as its input, and its output is an input to `Op.MaxPool`. 

#### 1.1.2 Rewrite Rules

ONNXSat contains the following operation fusion rules, which were chosen based on common patterns in the evaluation model:

- Conv + BatchNorm ->  FusedConvBatchNorm
- Conv + BatchNorm + Relu -> FusedConvBatchNorm
- Conv + Relu -> FusedConvActivation
- Conv + Clip -> FusedConvActivation
- Gemm + Relu -> FusedGemm


To add rules, we define a ruleset for Egglog and register rules to the ruleset. For example, the rule to rewrite a Gemm + Relu sequence to a FusedGemm operator is defined in [rewrites.py](eggie/rewrites.py) as:

In [ ]:
fusion_ruleset = ruleset(name="fusion_ruleset")

@fusion_ruleset.register
def fuse_gemm_relu(
    gemm_name: TensorId,
    gemm_in: Vec[TensorId],
    alpha: f64,
    beta: f64,
    transA: i64,
    transB: i64,
    gemm_out: TensorId,
    relu_name: TensorId,
    relu_out: TensorId,
):
    yield rewrite(
        Op.Relu(
            relu_name,
            Op.Gemm(
                gemm_name,
                GemmAttrs(alpha, beta, transA, transB),
                gemm_in,
                gemm_out,
            ),
            relu_out,
        )
    ).to(
        Op.FusedGemm(
            gemm_name,
            TensorId("com.microsoft"),
            FusedGemmAttrs(
                "Relu", f64(0.0), f64(0.0), f64(0.0), alpha, beta, transA, transB
            ),
            gemm_in,
            relu_out,
        )
    )

This snippet instructs Egglog to rewrite an expression containing an `Op.Relu` e-node receiving the result of an `Op.Gemm` node to an `Op.FusedGemm` expression.

Egglog applies all the rewrite rules that match expressions in a given computation graph until it reaches saturation and extracts the best program according to the cost model. 

The cost model for ONNXSat is simple: all the operators have the same Egglog default cost of 1. However, since the rewrite rules fuse multiple operators into one, Egglog will select the single fused operator as the best e-node from an e-class, as that will yield a lower cost expression than one with multiple operators. 

#### 1.1.3. Egglog to ONNX Conversion

ONNXSat converts the extracted Egglog expression back into ONNX format using `Lexer` and `EggParser` classes defined in [lexer.py](eggie/lexer.py) and [parser.py](eggie/parser.py) respectively. The lexer splits the expression into tokens that the parser uses to create ONNX nodes. 

The `EgglogTokenKind` enum defined in [lexer.py](eggie/lexer.py) represents the expected input tokens:

In [ ]:
class EgglogTokenKind(Enum):
    AVG_POOL_ATTRS = auto()
    BATCH_NORM_ATTRS = auto()
    COMMA = auto()
    CONV_ATTRS = auto()
    DOT = auto()
    EQUALS = auto()
    EOF = auto()
    FLOAT_LITERAL = auto()
    FUSED_CONV_ATTRS = auto()
    FUSED_GEMM_ATTRS = auto()
    GEMM_ATTRS = auto()
    LEFT_PARENTHESIS = auto()
    LEFT_SQUARE_BRACKET = auto()
    MAX_POOL_ATTRS = auto()
    INTEGER_LITERAL = auto()
    OP = auto()
    QUANTIZE_LINEAR_ATTRS = auto()
    RIGHT_PARENTHESIS = auto()
    RIGHT_SQUARE_BRACKET = auto()
    STRING_LITERAL = auto()
    TENSOR_ID = auto()
    TENSOR_TYPE = auto()
    VARIABLE_NAME = auto()
    VEC = auto()

And the file contains a mapping from string values to the different tokens:

In [ ]:
str_to_token = {
    "AveragePoolAttrs": EgglogTokenKind.AVG_POOL_ATTRS,
    "BatchNormAttrs": EgglogTokenKind.BATCH_NORM_ATTRS,
    "ConvAttrs": EgglogTokenKind.CONV_ATTRS,
    "FusedConvAttrs": EgglogTokenKind.FUSED_CONV_ATTRS,
    "FusedGemmAttrs": EgglogTokenKind.FUSED_GEMM_ATTRS,
    "GemmAttrs": EgglogTokenKind.GEMM_ATTRS,
    "MaxPoolAttrs": EgglogTokenKind.MAX_POOL_ATTRS,
    "Op": EgglogTokenKind.OP,
    "QuantizeLinearAttrs": EgglogTokenKind.QUANTIZE_LINEAR_ATTRS,
    "TensorId": EgglogTokenKind.TENSOR_ID,
    "TensorType": EgglogTokenKind.TENSOR_TYPE,
    "Vec": EgglogTokenKind.VEC,
    "=": EgglogTokenKind.EQUALS,
    "[": EgglogTokenKind.LEFT_SQUARE_BRACKET,
    "]": EgglogTokenKind.RIGHT_SQUARE_BRACKET,
    "(": EgglogTokenKind.LEFT_PARENTHESIS,
    ")": EgglogTokenKind.RIGHT_PARENTHESIS,
    ",": EgglogTokenKind.COMMA,
    ".": EgglogTokenKind.DOT,
}

The `Lexer` class has a `next_token()` method that provides the next unconsumed input token to the `EggParser` class. 


The `EggParser` parses tokens from the `Lexer` until there are no more tokens left to process. When it receives an `EgglogTokenKind.OP` token, it invokes the relevant method to parse an operator. 

For example, if the input contains `Op.MatMul`, it invokes the `_parse_matmul()` method to create a `MatMul` node. This method is defined as:

In [ ]:
def _parse_matmul(self) -> NodeProto:
    return self._get_node(
        "MatMul",
        input_types=[EgglogTokenKind.TENSOR_ID, EgglogTokenKind.TENSOR_ID],
        output_types=[EgglogTokenKind.TENSOR_ID],
    )

All operator parsing functions use the `_get_node()` helper method defined in the `Parser` class to create an ONNX node. This method has the signature:

In [ ]:
def _get_node(
    self,
    op_type: str,
    input_types: List[EgglogTokenKind],
    output_types: List[EgglogTokenKind],
    attributes: List[Attribute] = [],
    has_domain: bool = False,
) -> NodeProto

The `_get_node()` method parses the inputs, outputs, and any attributes for a given operator and creates an ONNX node. Additionally, if the operator is not a standard ONNX operator but has been contributed by another and so has a custom domain [tk - rewrite], the method will parse the domain. 

`EggParser` returns a list of all the ONNX nodes when there are no more inputs to consume. 

#### 1.1.4. Putting it all together

The `ModelOptimizer` class defined in [model_optimizer.py](model_optimizer.py) connects the different pieces together in its `run()` method, which operates on a given computation graph:

In [ ]:
def run(self, ...) -> ModelProto:
    ....    
    egg_expr = Converter.to_Egglog(graph)
    egraph = EGraph(save_Egglog_string=True)
    egg_expr = egraph.let("expr", egg_expr)
    ...
    # Run equality saturation until the given limit or all the rules have been applied
    egraph.run(limit=1000, ruleset=fusion_ruleset)
    extracted = egraph.extract(egg_expr)
    ...
    converted_onnx_nodes = Converter.to_onnx(extracted)
    new_model = self._update(new_model, converted_onnx_nodes)

After converting back to ONNX, there is an extra processing step for the `FusedConvBatchNorm` operator. Unlike the `FusedConvActivation` operator which uses a Microsoft-provided `FusedConv` [7] ONNX operator in the `com.microsoft` domain, there is no public ONNX operator that combines a convolution and batch normalization.

I implement this fusion in the `_process_conv_bn` method of the `ModelOptimizer` class, creating new weights and bias tensors:

In [ ]:
def _process_conv_bn(self, node: gs.Node) -> gs.Node:
    ... # Skip input parsing
    
    # Fold the batch normalization into the convolution and update the conv bias and weights 
    adjusted_scale = bn_scale / np.sqrt(bn_input_var + epsilon)

    new_w = conv_w * adjusted_scale[:, None, None, None]
    new_b = (conv_bias - bn_input_mean) * adjusted_scale + bn_bias

    ... # Skip new node creation. 

Hubens [8] explains this fusion in more detail, but in essence, because convolution and batch normalization are linear operations, they can be combined to create a single linear operation. [tk - anything else?]

The result of the `ModelOptimizer.run()` function is an optimized ONNX model containing fused operators. 

### 1.2. Evaluation

To evaluate ONNXSat, I compared the inference latency for its generated model with the optimizations from the ONNX Runtime Engine against a baseline of no optimizations on five CNN models, four of which are standard models: MobileNetV2 [9], MobileNetV4 [10], ResNet18 [11], and ShuffleNetV2 [12] trained on the ImageNet 2012 dataset [13].

The final model is a simple classifier from a previous lab trained on the CIFAR-10 [14] dataset and defined as:

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt


class SimpleClassifier(nn.Module):
    """Simple CNN adapted from 'PyTorch: A 60 Minute Blitz'."""

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


Figure 2 below illustrates the result of optimizing the simple classifier using ONNXSat. The graph on the left is the original form and the one on the right is the optimized one obtained by fusing operators. 

<figure style="text-align: center; margin: 5px">
  <p float="left"> 
  <img
  src="report/simple_classifier_orig.png"
  style="height:20%;">
    <img
  src="report/simple_classifier_fused.png"
  style="height:20%; margin-left: 60px; margin-bottom: 50px">
  <figcaption> <b>Figure 2. </b> Result of applying ONNXSat on the simple classifier model. The image on the left is the original model and the optimized ONNXSat model is on the right.  </figcaption>
</figure>


All experiments were run on an Apple MacBook Air with an Apple M2 chip running macOS 15 with 16GB memory. The reported results use the median latency value obtained by measuring the latency over 3 runs, each performing inference on 2000 samples from the relevant dataset. The accuracy after each run is also verified to be unchanged. 


The `BenchmarkTool` class in [bench.py](bench/bench.py) contains methods for executing all the benchmarks. The code to execute this experiment is defined as:

In [ ]:
# Named tuples that serve as types for the benchmarking
Accuracy = namedtuple("Accuracy", "top_one top_five")
Latency = namedtuple("Latency", "mean median percentile_95")
Benchmark = namedtuple("Benchmark", "num_samples accuracy latency throughput")


class BenchMarkCategory(Enum):
    BASELINE = auto()
    ONNX_SAT = auto()
    ONNX_OPT = auto()
    ONNX_SAT_AND_OPT = auto()


category_to_display_name = {
    BenchMarkCategory.BASELINE: "Baseline - No Optimization",
    BenchMarkCategory.ONNX_SAT: "OnnxSAT",
    BenchMarkCategory.ONNX_OPT: "Onnx Optimizations",
    BenchMarkCategory.ONNX_SAT_AND_OPT: "OnnxSAT + OnnxOptimizations",
}

def experiment_one(self) -> None:
    results: Dict[str, Dict[BenchMarkCategory, Benchmark]] = {}
    results_dir = f"{self._results_dir}/1"

    if not os.path.exists(results_dir):
        os.makedirs(results_dir)

    for model_name, orig_model in self._model_name_to_file.items():        
        # Apply optimizations in offline mode, so optimization overhead does not affect inference measurement 
        # https://onnxruntime.ai/docs/performance/model-optimizations/graph-optimizations.html#onlineoffline-mode
        onnx_optimized_model = (
            f"{self._onnx_optimized_models_dir}/{model_name}.onnx"
        )

        # This method enables all available ONNX optimizations and stores the resulting ONNX file in the file path above
        self.apply_onnx_opt_offline(model_name, orig_model, onnx_optimized_model)

        new_model = None
        with open(orig_model, "rb"):
            model_optimizer = ModelOptimizer(
                orig_model, self._data_dir, results_dir
            )
            new_model = model_optimizer.run()

        # Offline-mode optimizations to measure the inference latency see of an ONNXSat + ONNX Runtime Engine-optimized model
        eqsat_and_onnx_optimized_model = (
            f"{self._onnx_optimized_models_dir}/{model_name}-eqsat.onnx"
        )
        self.apply_onnx_opt_offline(
            model_name, new_model, eqsat_and_onnx_optimized_model
        )

        category_to_model = {
            BenchMarkCategory.ONNX_SAT: new_model,
            BenchMarkCategory.ONNX_OPT: onnx_optimized_model,
            BenchMarkCategory.ONNX_SAT_AND_OPT: eqsat_and_onnx_optimized_model,
            BenchMarkCategory.BASELINE: orig_model,
        }

        category_results = {}
        warmup_loader, num_labels = self._get_loader_and_num_labels(
            model_name, num_samples=1000
        )
        loader, num_labels = self._get_loader_and_num_labels(
            model_name, num_samples=2000
        )

        for category, model in category_to_model.items():
            sess_options = ort.SessionOptions()
            sess_options.graph_optimization_level = (
                ort.GraphOptimizationLevel.ORT_DISABLE_ALL # Disable online-mode optimizations
            )
            session = ort.InferenceSession(model, sess_options=sess_options)

            # Warm up
            self._measure(session, warmup_loader, num_labels)

            # Actual Measurment
            category_results[category] = self._measure(
                session, loader, num_labels, num_runs=3
            )

        results[model_name] = category_results

    self._visualize_latency_and_throughput(1, category_to_model.keys(), results)

The `_measure()` method, which runs inference and reports the benchmark values for each model across all experiments is defined as:

In [ ]:
def _measure(
    self,
    session: ort.InferenceSession,
    dataloader: DataLoader,
    num_labels=1000,
    num_runs=1,
) -> Benchmark:
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    all_preds = []
    all_labels = []

    latencies = []

    overall_start_time = time.perf_counter()

    for _ in range(num_runs):
        run_latencies = []
        for inputs, labels in tqdm(
            dataloader, desc="Inference Progress", unit="batch"
        ):
            inputs_numpy = inputs.numpy()
            labels_numpy = labels.numpy()

        # Run inference
        start_time = time.perf_counter()
        outputs = session.run([output_name], {input_name: inputs_numpy})
        end_time = time.perf_counter()
        
        run_latencies.append(end_time - start_time)

        all_preds.append(outputs[0])
        all_labels.append(labels_numpy)

        latencies.extend(run_latencies)

    overall_end_time = time.perf_counter()
    throughput = len(latencies) / (overall_end_time - overall_start_time)

    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    top1_accuracy = top_k_accuracy_score(
            all_labels, all_preds, k=1, labels=np.arange(num_labels)
        )
    top5_accuracy = top_k_accuracy_score(
            all_labels, all_preds, k=5, labels=np.arange(num_labels)
        )

 
    accuracy = Accuracy(top1_accuracy, top5_accuracy)

    mean_latency = np.mean(latencies)
    median = np.median(latencies)
    percentile_95 = np.percentile(latencies, 95)

    latency = Latency(mean_latency, median, percentile_95)

    return Benchmark(len(latencies), accuracy, latency, throughput)



#### 1.2.1. Results

Figure 3 shows the median latency speedup for different optimization methods against a baseline of no optimizations. The optimization passes obtain a speedup of 1.03x to 1.24x for all models over the baseline. 


<figure style="text-align: center; margin: 5px">
  <img
  src="report/1_latency.png"
  alt="Image containing five grouped bars demonstrating the speedup of different optimizations against a baseline with no optimizations"
  style="width: 50%; height: auto;">
  <figcaption> <b>Figure 3. </b> Median latency speedup against a baseline with no optimizations for five CNN models. </figcaption>
</figure>


Figure 4 displays the throughput of each optimization method against the baseline. The throughput does not always correlate with the latency values, but we still observe throughput gains from operator fusion for the state-of-the-art models. 

<figure style="text-align: center; margin: 5px">
  <img
  src="report/1_throughput.png"
  alt="Image containing five grouped bars demonstrating the speedup of different optimizations against a baseline with no optimizations"
  style="width: 50%; height: auto;">
  <figcaption> <b>Figure 4. </b>Throughput against a baseline with no optimizations for five CNN models. </figcaption>
</figure>


Table 1 presents the operator patterns exploited in each model for fusion to yield the results in the previous section.  The values in this table were obtained by profiling the ONNX models before and after conversion using an open-source ONNX profiling tool [15]. 

| Model          | % of Operator Fused|Fusion Pattern(s)       | Original Number of MACs | ONNXSat-Optimized Number of MACs | MAC Reduction | Original Number of Params | ONNXSat-Optimized Number of Params | Params Reduction | Original Memory Usage (Bytes) | ONNXSat-Optimized Memory Usage (Bytes) | Memory Usage Reduction|
|----------------|----------------------|----------------------|--------------------------|-----------------------------------|---------------|-----------------------|-----------------------|-----------------------|-----------------------|--------------------------------|--------------------------------|
|    [Simple Classifier](data/models/simple_classifier.onnx)            | 57.1%  | Conv -> Relu & Gemm -> Relu                  |    671,050                      |              664,858                     |        0.9%       |  62,006    | 62,006   | 0% |  308,032                |           283,264                     | 8% 
|   [Resnet18](data/models/resnet18.onnx)     |    22.5%   |        Conv -> Relu              |      1,821,452,264                    |        1,819,896,808                           |         0.085%      | 11,684,712    | 11,684,712   | 0% |  69,727,552                |            63,505,728                    | 8.9% 
|   [ShuffleNet-v2](data/models/shufflenet-v2.onnx)       |  63.64%    |      Conv -> BatchNorm & Conv -> BatchNorm -> Relu                |       207,580,776                   |                     147,586,744              |        28.9%       |  2,294,784     |  2,270,514     |  1%    |   84,371,520             |                  56,768,584              | 32.7% 
|     [MobileNetV2](data/models/mobilenetv2.onnx)    |  54.69%     |     Conv -> Clip                 |       319,949,192                   |        307,737,608                            |     3.82%  |   3,487,887  |   3,487,887   |   0%      |       117,982,392                |              69,135,776                   | 41.4%
|   [MobileNetV4](data/models/mobilenetv4.onnx)   | 50.85%    |      Conv -> Relu                |               189,867,112           |                188,147,304                   |            0.9%  |  3,761,480     |  3,761,480  |  0%  |     30,297,536           |              23,418,304                   | 22.7%


#### 1.2.2. Discussion

The results in the previous section demonstrate the benefits of operator fusion, causing a reduction in inference latency, number of multiply-accumulate (MAC) operations and params, and overall memory usage. We also see that some operator fusions are more effective than others. For example, the Conv -> BatchNorm fusion in ShuffleNet-V2 led to the highest MAC reduction percentage of 28.9%. The Conv -> Clip fusion also proves effective, reducing memory usage by ~41% compared to the original model. The Conv -> Relu fusion, however, only leads to marginal reductions in MACs, but still 8-22% reduction in memory usage.

##### 1.2.2.1. Latency and Throughput Observations

One notable observation from the inference latency plot is that the speedup across models does not directly correlate with reduction in the number of MACs or memory usage. For example, ShuffleNet-V2 achieved a 28.9% reduction in MACs but did not experience the highest latency gain. Instead, ResNet18, which achieved only a modest MAC reduction of 0.085%, exhibited the highest latency speedup of 1.24x.

This discrepancy highlights both the importance and challenge of profiling and benchmarking performance metrics. While profiling data such as the percentage of operators fused and the reduction in the number of MACs might suggest that ShuffleNetV2 would lead to the greatest latency improvement, real-world results do not align with this expectation. Other factors such as hardware-specific optimizations and cache effects can affect latency, and may have impacted these measurements despite my best efforts to reduce their effect. 

We also see throughput gains for the state-of-the-art models that are not directly proportional to their MAC reduction. ONNXSat matches the throughput for the Simple Classifier and we see a reduction for the other optimization methods. However, like with latency, the throughput measurements can be affected by external factors, and the reduction is not significant enough to draw any conclusions from. 

##### 1.2.2.2. Memory Usage and Parameters
The results show that operator fusion also reduces memory usage, with up to 41.4% reduction for MobileNetV2, which is computed for a layer as: `Layer memory usage = Weight tensor memory + Output tensor memory`. We see a reduction in memory usage without reducing the number of params, just by fusing operators. The Conv -> BatchNorm fusion reduces the number of params since the BatchNorm operator gets extra parameters, but we can deduce from the other result that the fusion contributes more to the memory usage reduction. [tk - review]

### 1.3. Conclusion

Overall, this experiment has provided a quantitative analysis of the effect of operator fusion on ONNX models. By reducing computational requirements and memory usage, operator fusion supports more efficient deployment of models on resource-constrained hardware without reducing the model accuracy or requiring retraining. We also see that the effectiveness of operator fusion depends on the model architecture and the specific operators fused. Future work could explore fusing other patterns like Concat->Reshape, Shape->Gather, and Conv->Add.  

## 2. How Sparse Can We Go? 

My second experiment explores pruning weights for optimizing deep neural networks. Keeping with the theme of this project, it aims to answer the question:

>  How much sparsity can we introduce into a network without needing to retrain it?

Introducing sparsity into a pre-trained model inevitably affects its accuracy. However, this experiment sets a threshold for tolerable accuracy loss and determines how sparse we can make our CNN models while keeping Top-5 accuracy loss within the tolerable threshold.    

For this experiment, I referred to the work by Ashouri et al. [16], which introduces three retraining-free methods for pruning convolutional neural networks. The methods differ in how they determine the thresholds they use to set weights with absolute values below the threshold to zero.

In the *Flat* method, the same threshold is applied across all the layers. This threshold is determined by profiling the layers in the network to determine the one with the smallest range of weights. This range, referred to as the *span*, serves as an upper bound for the threshold. The final threshold is calculated as a fraction of the span, which can be adjusted as needed.


The *Triangular* method is based on the fact that the early convolution layers in many state-of-the-art CNNs have fewer parameters than the later layers. Therefore, it selects thresholds such that the sparsity increases as we go from early layers to later ones. This mitigates accuracy loss from pruning too many neurons in earlier layers. The sparsification threshold differs per layer and falls within a limit based on the span of the first convolution layer and last fully-connected layer.

Lastly, the *Relative* method selects a threshold per layer based on the distribution of weights in the layer, where the threshold $T$ of a layer $L$ is the $i^{th}$ percentile of weights in the layer. Weights with absolute values less than or equal to $T$ are set to zero. This addresses a limitation of the previous methods, which may perform poorly for CNNs whose layers exhibit a more complex structure than a linear expansion. For example, some networks replace bigger layers with groups of small ones, which may lead to smaller layers being pruned too aggressively in the flat and triangular methods. 



This experiment considers the Relative method, implemented in the `_sparsify()` method of the `ModelOptimizer` class:

In [ ]:
def _sparsify(self, model: ModelProto, sparsity_fraction: float) -> ModelProto:
    initializer_by_name = {init.name: init for init in model.graph.initializer}

    for node in model.graph.node:
        if node.op_type == "Conv" or node.op_type == "FusedConv":
            # Get the weight tensor
            weights_init = initializer_by_name.get(node.input[1])
            weights_tensor = onnx.numpy_helper.to_array(weights_init)
            orig_shape = weights_tensor.shape
            weights_tensor = weights_tensor.flatten()
            
            # Determine the dynamic threshold based on the sparsity fraction
            threshold = np.percentile(
                np.abs(weights_tensor), sparsity_fraction * 100
            )
            
            weights_tensor[np.abs(weights_tensor) < threshold] = 0
            weights_tensor = weights_tensor.reshape(orig_shape)
            # Replace the weight tensor with the sparsified value
            weights_init.CopyFrom(
                onnx.numpy_helper.from_array(weights_tensor, weights_init.name)
            )
    return model


For a given a sparsity fraction which represents a percentile, `_sparsify()` sets all convolution weights that are less than or equal to the percentile value to zero.  

### 2.1. Evaluation

This section examines the impact of introducing sparsity through the relative method on model accuracy. The evaluation uses percentile values ranging from 10% to 85%, with steps of 5%, each tested on 5000 samples. I set 5% as the tolerable accuracy loss threshold for this experiment.

#### 2.1.1. Experimental Setup
The `BenchMarkTool` class in [bench.py](bench/bench.py) contains the method for running this experiment:

In [ ]:
def experiment_two(self) -> None:
    
    sparsity_ratios = [ratio.item() for ratio in np.arange(0.1, 0.9, 0.05)]

    top_one_accuracy_loss_per_ratio: Dict[str, Dict[float, float]] = defaultdict(
        dict
    )
    top_five_accuracy_loss_per_ratio: Dict[str, Dict[float, float]] = defaultdict(
        dict
    )

     # ... Omitted results storage logic 

    for model_name, orig_model in self._model_name_to_file.items():

        loader, num_labels = self._get_loader_and_num_labels(
            model_name, num_samples=5000
        )

        # Determine baseline 
        session = ort.InferenceSession(orig_model)
        baseline_accuracy = self._measure(session, loader, num_labels).accuracy
        base_top_one = baseline_accuracy.top_one
        base_top_five = baseline_accuracy.top_five
        
        
        for ratio in sparsity_ratios:
            new_model = None
            with open(orig_model, "rb"):
                model_optimizer = ModelOptimizer(
                    orig_model, self._data_dir, results_dir
                )
                new_model = model_optimizer.run(
                    sparsity_ratio=ratio, prune_only=True
                )

            session = ort.InferenceSession(new_model)
            accuracy = self._measure(session, loader, num_labels).accuracy
            
            top_one_accuracy_loss_per_ratio[model_name][ratio] = (
                np.round(accuracy.top_one - base_top_one, 2) * 100
            ).item()
            top_five_accuracy_loss_per_ratio[model_name][ratio] = (
                np.round(accuracy.top_five - base_top_five, 2) * 100
            ).item()

            # Optimization: Stop processing if Top-5 accuracy drops to zero, as it won't get better.
            if round(accuracy.top_five, 2) == 0:
                break
    # ...

    self._visualize_accuracy_drop("Relative", 5, top_five_accuracy_loss_per_ratio)
    self._visualize_accuracy_drop("Relative", 1, top_one_accuracy_loss_per_ratio)

The `experiment_2()` method uses the `_measure()` method presented earlier to run inference, and uses a helper `_visualize_accuracy_drop()` method, which receives a dictionary of the computed values, to plot the results:

In [ ]:
def _visualize_accuracy_drop(
    self, method: str, top: int, results: Dict[str, Dict[float, float]]
) -> None:
    # Generate dynamic colors and markers
    num_models = len(results.keys())
    colors = plt.cm.tab10(np.linspace(0, 1, num_models))
    markers = ["o", "s", "^", "D", "P", "X", "*"]

    for i, (model, result_vals) in enumerate(results.items()):
        plt.plot(
            [0] + list(result_vals.keys()),
            [0] + list(result_vals.values()),
            linestyle="--",
            color=colors[i % len(colors)],
            marker=markers[i % len(markers)],
            label=model,
        )

    plt.xlabel("Model Sparsity (%)")
    plt.ylabel(f"Top-{top} Accuracy Drop (%)")
    plt.title(f"Top-{top} Accuracy Drop vs Model Sparsity ({method} Method)")
    
    # Draw a red line across the -5% threshold on the y axis
    plt.axhline(-5, color="red", linewidth=1, linestyle="--")
    
    plt.xticks(range(0, 90, 10))
    plt.xlim(left=0)

    plt.legend(loc="upper right", fontsize=10)
    plt.grid(True, linestyle="-", alpha=0.5)

    plt.minorticks_on()
    plt.grid(which="minor", linestyle="-", linewidth=0.5, alpha=0.2)

    plt.tick_params(which="minor", length=0)
    plt.tick_params(axis="both", which="major", direction="in")
    
    plt.savefig(f"{self._results_dir}/2/{method.lower()}_{top}_accuracy_loss.png")
    plt.close()

#### 2.1.2. Results


The Top-5 accuracy drop at increasing sparsity levels for the different models is shown in Figure 5 below:

<figure style="text-align: center; margin: 5px">
  <img
  src="report/relative_5_accuracy_loss.png"
  alt="Top-5 accuracy drop for different models using the relative method at different sparsity levels."
  style="width: 50%; height: auto;">
  <figcaption> <b>Figure 5.</b> Top-5 accuracy drop at different sparsity levels for the five models. The red dotted line indicates the 5% threshold.</figcaption>
</figure>


Figure 6 presents the Top-1 accuracy results below:

<figure style="text-align: center; margin: 5px">
  <img
  src="report/relative_1_accuracy_loss.png"
  alt="Top-1 accuracy drop for different models using the relative method at different sparsity levels."
  style="width: 50%; height: auto;">
  <figcaption> <b>Figure 6.</b> Top-1 accuracy drop at different sparsity levels for the five models. </figcaption>
</figure>



#### 2.1.3 Discussion

[tk - add table for model accuracy and stuff]

The results validate the effect of the architecture of the model on the accuracy loss caused by introducing sparsity. Figure 5 shows that we can gain up to 30% sparsity (1.43x compression factor) for ShuffleNet-V2 and ResNet18 while keeping Top-5 accuracy loss within 5%. For the simple classifier, we can gain up to 70% sparsity (3.33x compression factor) while keeping accuracy loss within the threshold. However, MobileNetV2 and MobileNetV4 do not play well with the relative method for sparsification, dropping below 5% accuracy loss at 14% and 1% sparsity respectively.

This is because of the structure of the models. In the MobileNet architectures, the size of the convolution layers increases linearly, and the relative method over-prunes the earlier layers. This is in contrast to ShuffleNet-V2, for example, where the size is more consistent across layers.


Figure 6 shows a similar trend for the Top-1 accuracy, which remains within the 5% threshold as we gain up to 26% and 30% sparsity for ShuffleNetV2 and ResNet18 respectively. The simple classifier can gain 50% sparsity (2x compression) and still be within 5% of its original accuracy.


### 2.2. Conclusion
This experiment has evaluated the relative method for sparsification without retraining. Sparsification without retraining provides the benefits of sparsification when there is no access to training data or retraining is expensive. Using the relative method, we can get up to 30% sparsity with less than 5% drop in Top-5 inference accuracy. However, the performance of a sparsification method depends on the model architecture, and we have seen how the relative method is not effective for the MobileNet architecture. Future work can explore other implementing other sparsification methods and training a classifier to predict what method to use based on the model architecture, as done by Ashouri et al [16].

## 3. What Does Sparsity Give Us?

Sparsifying a model's weights is more effective when the presence of zero values can be exploited on hardware. In this experiment, I examine the impact of sparsifying a model on its performance. Ashouri et al [16] claim in their paper that sparsifying convolution weights instead of input data, for example, "alleviates the need for specialized hardware accelerators to exploit the sparsity." 

This experiment measures the performance of models after sparsifying their weights. First, I adapt code from the official ONNX runtime codebase [17] to convert sparsified weights in a model to ONNX `SparseTensor` representations. This code is located in the [sparsify_initializers.py](sparsify_initializers.py) file.

In [ ]:
# -------------------------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.
# --------------------------------------------------------------------------
# This script opens an existing model in onnx format and attempts to
# move initializers from model.graph.initializer field to model.graph.sparse_initializer field
# and convert them into ONNX COO flat index format.

# Adapted code from : https://github.com/microsoft/onnxruntime/blob/main/tools/python/sparsify_initializers.py

import logging
import sys
from typing import Tuple

import numpy as np
import onnx
from onnx import ModelProto, SparseTensorProto, TensorProto, numpy_helper

logger = logging.getLogger(__name__)
log_handler = logging.StreamHandler(sys.stdout)
log_handler.setFormatter(logging.Formatter("%(filename)20s: %(message)s"))
logging_level = logging.ERROR
log_handler.setLevel(logging_level)
logger.addHandler(log_handler)
logger.setLevel(logging_level)

real_types = {int(TensorProto.FLOAT), int(TensorProto.DOUBLE)}

def convert_tensor_to_sparse(tensor, sparsity_threshold, tolerance):  # type: (TensorProto, float, float) -> Tuple[SparseTensorProto, float]
    """returns a tuple of sparse_tensor and sparsity level"""
    values = []
    indices = []
    nnz_count = 0
    tensor_data = numpy_helper.to_array(tensor).flatten()
    data_len = len(tensor_data)

    if tensor_data.dtype in real_types:
        for index in range(data_len):
            el = tensor_data[index]
            if abs(el) <= tolerance: 
                values.append(el)
                indices.append(index)
                nnz_count += 1
    else:
        for index in range(data_len):
            el = tensor_data[index]
            if el != 0:
                values.append(el)
                indices.append(index)
                nnz_count += 1

    sparsity = 1.0 - float(nnz_count) / data_len

    ind_data_type = TensorProto.INT64
    ind_dtype = np.int64
    ind_len = len(indices)

    sparsity = np.round(sparsity, 2)
    if sparsity < sparsity_threshold:
        return (object(), sparsity)

    # create np array and cast data to the appropriate type
    np_values = np.array(values).astype(tensor_data.dtype)
    # create np array and cast data to the inferred index type
    np_indices = np.array(indices).astype(ind_dtype)

    values_tensor = onnx.helper.make_tensor(
        tensor.name, tensor.data_type, [len(values)], np_values.tobytes(), raw=True
    )

    indicies_tensor = onnx.helper.make_tensor(
        tensor.name + "_indicies",
        ind_data_type,
        [ind_len],
        np_indices.tobytes(),
        raw=True,
    )

    sparse_tensor = onnx.helper.make_sparse_tensor(
        values_tensor, indicies_tensor, tensor.dims
    )
    return (sparse_tensor, sparsity)


def convert_initializers(model, sparsity_threshold, tolerance):  # type: (ModelProto, float, float) -> ModelProto
    graph = model.graph
    converted_sparse = []
    remaining_initializers = []
    for initializer in graph.initializer:
        if initializer.data_type == TensorProto.BOOL:
            remaining_initializers.append(initializer)
            continue
        sparse_tensor, sparsity = convert_tensor_to_sparse(
            initializer, sparsity_threshold, tolerance
        )

        if sparsity >= sparsity_threshold:
            converted_sparse.append(sparse_tensor)
        else:
            remaining_initializers.append(initializer)


    graph.sparse_initializer.extend(converted_sparse)
    del graph.initializer[:]
    graph.initializer.extend(remaining_initializers)
    return model



### 3.1. Evaluation

To evaluate the impact of sparsification, I choose the highest sparsity value within the 5% Top-5 accuracy drop threshold for each model and record the inference latency and throughput.


#### 3.1.1. Experimental Setup

The method below defined in `BenchmarkTool` performs the experiment:

In [ ]:
def experiment_three(self) -> None:
    results: Dict[str, Dict[BenchMarkCategory, Benchmark]] = {}


    # Max sparsity ratio using the results in experiment 2 that keeps the results within the 5% accuracy drop threshold
    max_sparsity_ratio_per_model = {
        "simple_classifier": 0.74,
        "resnet18": 0.32,
        "mobilenetv2": 0.14,
        "mobilenetv4": 0.01,
        "shufflenet-v2": 0.28,
    }

    for model_name, orig_model in self._model_name_to_file.items():
        sparsity_ratio = max_sparsity_ratio_per_model[model_name]

        onnx_optimized_model = (
            f"{self._onnx_optimized_models_dir}/{model_name}.onnx"
        )
        self.apply_onnx_opt_offline(model_name, orig_model, onnx_optimized_model)
        
        new_model = None
        with open(orig_model, "rb"):
            model_optimizer = ModelOptimizer(
                orig_model, self._data_dir, results_dir
            )
            new_model = model_optimizer.run(sparsity_ratio=sparsity_ratio)
        
        category_to_model = {
            BenchMarkCategory.ONNX_SAT: new_model,
            BenchMarkCategory.ONNX_OPT: onnx_optimized_model,
            BenchMarkCategory.BASELINE: orig_model,
        }
        category_results = {}
        
        warmup_loader, num_labels = self._get_loader_and_num_labels(
            model_name, num_samples=1000
        )
        loader, num_labels = self._get_loader_and_num_labels(
            model_name, num_samples=2000
        )
        
        for category, model in category_to_model.items():
            sess_options = ort.SessionOptions()
            sess_options.graph_optimization_level = (
                ort.GraphOptimizationLevel.ORT_DISABLE_ALL
            )
            session = ort.InferenceSession(model, sess_options=sess_options)
            
            # Warm up
            self._measure(session, warmup_loader, num_labels)
            
            # Actual Measurment
            category_results[category] = self._measure(
                session, loader, num_labels, num_runs=3
            )
        
        # Format the map keys, as they form the x-axis labels of the plot
        results[f"{model_name}\n ({round(sparsity_ratio*100)}% sparsity)"] = (
            category_results
        )

    self._visualize_latency_and_throughput(3, category_to_model.keys(), results)


#### 3.1.2. Results
Figure 7 shows the speedup gained by combining sparsification and operator fusion. Only the blue bar representing ONNXSat runs the pruned model. The baseline and ONNX optimization measurements are taken from running the unpruned model. 


<figure style="text-align: center; margin: 5px">
  <img
  src="report/3_latency.png"
  alt="Top-1 accuracy drop for different models using the relative method at different sparsity levels."
  style="width: 50%; height: auto;">
  <figcaption> <b>Figure 7. </b> Median latency speedup after sparsification against a baseline with no optimizations for five CNN models. </figcaption>
</figure>

Figure X shows the throughput values:

<figure style="text-align: center; margin: 5px">
  <img
  src="report/3_throughput.png"
  alt="Top-1 accuracy drop for different models using the relative method at different sparsity levels."
  style="width: 50%; height: auto;">
  <figcaption> <b>Figure 8. </b> Throughput against a baseline with no optimizations for five CNN models. </figcaption>
</figure>




#### 3.1.3. Discussion

The results show that for the state-of-the-art models, we can gain performance benefits from sparsifying a model even without specialized hardware accelerators exploiting sparsity. We introduce the highest sparsity in the ResNet18 model (32%) and that gives us the highest performance gain of 1.4x, and 1.35x over the equivalent dense model with all ONNX runtime optimizations applied.   

Similarly, introducing sparsity increases the throughput of each model, with models with higher sparsity getting better throughput. The exception to this trend is the simple classifier, which achieves slightly worse latency and throughput than the unpruned version. However, as it is a small model. It is difficult to draw conclusions from it. [tk - rewrite]

### 3.2. Conclusion

Unfortunately, technical constraints due to the poor support for sparse tensors in the ONNX profiler and time constraints prevent me from conducting an analysis of the number of MACs and memory reduction gained by sparsifying the tensors, as done in Experiment 1. That is left as future work.  

Overall, the results show that without losing more than 5% of Top-5 accuracy, we can gain improve the latency of our model by up to 40%. 

## 4. Limitations and Future Work
The results show what we can gain by without retraining a model by implementing operator fusion and sparsification. However, there are a few directions this could go in for most robust experimentation:

- **Implementing more rewrite rules:** Despite the rewrite rules in ONNXSat exploiting most of the patterns in the evaluated models. However, there are more operator fusion rules that can be implemented for ONNX models, such as fusing Shape -> Gather sequences into one operation and doing the same for Concat -> Reshape.
- **Evaluating on larger models:** Egglog's extraction is slow and I ran out of memory for some larger models like ResNet-50, DenseNet, EfficientNet, InceptionV3, and newer vison transformer models. There is also room to experiment on 
- **Developing a better cost model for equality saturation**: Empirical measurement.
- **Implementing different sparsification methods**: Finally, there is room to implement the Flat and Triangle sparsification methods presented in the paper by Ashouri et al. [tk - cite], as well as develop a classifier to determine what sparsification method to use.  



## 5. Conclusion
This project aimed to explore what we can gain in the performance of neural network models without retraining a model....

## References

[1] R. Tate, M. Stepp, Z. Tatlock, and S. Lerner, ‘Equality saturation: a new approach to optimization’, in Proceedings of the 36th annual ACM SIGPLAN-SIGACT symposium on Principles of programming languages, in POPL ’09. New York, NY, USA: Association for Computing Machinery, Jan. 2009, pp. 264–276. doi: 10.1145/1480881.1480915.

[2] M. Willsey, C. Nandi, Y. R. Wang, O. Flatt, Z. Tatlock, and P. Panchekha, ‘egg: Fast and extensible equality saturation’, Proc. ACM Program. Lang., vol. 5, no. POPL, pp. 1–29, Jan. 2021, doi: 10.1145/3434304.

[3] Y. Yang, P. M. Phothilimtha, Y. R. Wang, M. Willsey, S. Roy, and J. Pienaar, ‘Equality Saturation for Tensor Graph Superoptimization’, Mar. 17, 2021, arXiv: arXiv:2101.01332. Accessed: Nov. 12, 2024. [Online]. Available: http://arxiv.org/abs/2101.01332

[4] J. Hartmann, G. He, and E. Yoneki, ‘Optimizing Tensor Computation Graphs with Equality Saturation and Monte Carlo Tree Search’, in Proceedings of the 2024 International Conference on Parallel Architectures and Compilation Techniques, Long Beach CA USA: ACM, Oct. 2024, pp. 40–52. doi: 10.1145/3656019.3689611.

[5] Z. Jia, O. Padon, J. Thomas, T. Warszawski, M. Zaharia, and A. Aiken, ‘TASO: optimizing deep learning computation with automatic generation of graph substitutions’, in Proceedings of the 27th ACM Symposium on Operating Systems Principles, Huntsville Ontario Canada: ACM, Oct. 2019, pp. 47–62. doi: 10.1145/3341301.3359630.

[6] S. Shanabrook, Egglog Python: A Pythonic Library for E-graphs. [Online]. Available: https://github.com/egraphs-good/egglog-python

[7] ‘onnxruntime/docs/ContribOperators.md at main · microsoft/onnxruntime’, GitHub. Available: https://github.com/microsoft/onnxruntime/blob/main/docs/ContribOperators.md

[8] Nathan Hubens, ‘Faster Inference - Batch Normalization Folding’, fast.ai Course Forums. [Online]. Available: https://forums.fast.ai/t/faster-inference-batch-normalization-folding/69161

[9] M. Sandler, A. Howard, M. Zhu, A. Zhmoginov, and L.-C. Chen, ‘MobileNetV2: Inverted Residuals and Linear Bottlenecks’, Mar. 21, 2019, arXiv: arXiv:1801.04381. doi: 10.48550/arXiv.1801.04381.

[10] D. Qin et al., ‘MobileNetV4 -- Universal Models for the Mobile Ecosystem’, Sep. 29, 2024, arXiv: arXiv:2404.10518. doi: 10.48550/arXiv.2404.10518.

[11] K. He, X. Zhang, S. Ren, and J. Sun, ‘Deep Residual Learning for Image Recognition’, in 2016 IEEE Conference on Computer Vision and Pattern Recognition (CVPR), Jun. 2016, pp. 770–778. doi: 10.1109/CVPR.2016.90.

[12] N. Ma, X. Zhang, H.-T. Zheng, and J. Sun, ‘ShuffleNet V2: Practical Guidelines for Efficient CNN Architecture Design’, Jul. 30, 2018, arXiv: arXiv:1807.11164. doi: 10.48550/arXiv.1807.11164.

[13] O. Russakovsky et al., ‘ImageNet Large Scale Visual Recognition Challenge’, Jan. 30, 2015, arXiv: arXiv:1409.0575. doi: 10.48550/arXiv.1409.0575.

[14] A. Krizhevsky, ‘Learning Multiple Layers of Features from Tiny Images’, 2009. [Online]. Available: https://api.semanticscholar.org/CorpusID:18268744

[15] ‘ThanatosShinji/onnx-tool: A parser, editor and profiler tool for ONNX models.’ Accessed: Jan. 21, 2025. [Online]. Available: https://github.com/ThanatosShinji/onnx-tool

[16] A. H. Ashouri, T. S. Abdelrahman, and A. Dos Remedios, ‘Retraining-free methods for fast on-the-fly pruning of convolutional neural networks’, Neurocomputing, vol. 370, pp. 56–69, Dec. 2019, doi: 10.1016/j.neucom.2019.08.063.

[17] ONNX Runtime developers, ONNX Runtime. (Nov. 2018). [Online]. Available: https://github.com/microsoft/onnxruntime





